In [ ]:
import pandas as pd

file_path = '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/blue_data_clean.csv'

df = pd.read_csv(file_path, index_col='fecha', parse_dates=True)

df.head()
